<a href="https://colab.research.google.com/github/Tarandeep97/Smart-radiologist/blob/master/hackcovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files


uploaded = files.upload()


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


# New Section

In [2]:
!kaggle datasets download -d tarandeep97/xraysnormalcovid

100% 152M/152M [00:04<00:00, 51.7MB/s]



In [3]:
!mkdir chest-xray-pneumonia

!unzip xraysnormalcovid.zip -d chest-xray-pneumonia

Archive:  xraysnormalcovid.zip
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_155.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_156.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_157.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_158.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person31_bacteria_160.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person33_bacteria_169.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person33_bacteria_172.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/covid/nejmc2001573_f1b.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/covid/nejmoa2001191_f4.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/covid/nejmoa2001

In [4]:
TRAIN_PATH = './chest-xray-pneumonia/covid-normal-pneumonia/train'
VAL_PATH = './chest-xray-pneumonia/covid-normal-pneumonia/val'
TEST_PATH = './chest-xray-pneumonia/covid-normal-pneumonia/test'

In [5]:
import pathlib
data_dir = pathlib.Path(TRAIN_PATH)
image_count = len(list(data_dir.glob('*/*')))
print(image_count)

478


In [6]:
#Helper libraries
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D,GlobalAveragePooling2D 
from keras.layers import Dense, Dropout, Softmax, Input, Flatten,BatchNormalization, Activation 
from keras.optimizers import adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


Using TensorFlow backend.


In [7]:
np.random.seed(1)

In [8]:
image_gen = ImageDataGenerator(
        samplewise_center=False,  
        featurewise_std_normalization=True,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip= True,  
        vertical_flip=True)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [9]:
BATCH_SIZE= 14
IMG_H = 224
IMG_W = 224
STEP_PER_EPOCH = np.ceil(image_count / BATCH_SIZE)

In [10]:
train_batches = image_gen.flow_from_directory(
                                 TRAIN_PATH,
                                 target_size=(IMG_H, IMG_W),
                                 classes=['covid','normal', 'bacterial','viral'],
                                 batch_size=BATCH_SIZE)
val_batches = image_gen.flow_from_directory(
                                 VAL_PATH,
                                 target_size=(IMG_H, IMG_W),
                                 classes=['covid','normal', 'bacterial','viral'],
                                 batch_size=BATCH_SIZE)
test_batches = image_gen.flow_from_directory(
                                 TEST_PATH,
                                 target_size=(IMG_H, IMG_W),
                                 classes=['covid','normal', 'bacterial','viral'],
                                 batch_size=BATCH_SIZE)


Found 478 images belonging to 4 classes.
Found 56 images belonging to 4 classes.
Found 28 images belonging to 4 classes.


In [18]:
def model(learning_rate):
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same", input_shape=(IMG_H,IMG_W,3)))
        model.add(Conv2D(filters=64, kernel_size=(5,5), activation="relu", padding="same"))
        model.add(MaxPool2D(pool_size=(2,2), padding='same'))
        model.add(Conv2D(filters=128, kernel_size=(5,5), activation="relu", padding="same"))
        model.add(MaxPool2D(pool_size=(2,2), padding='same'))
        model.add(Dropout(rate=0.4))

        model.add(Flatten())

        model.add(Dense(1024, activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(rate=0.4))
        model.add(Dense(4, activation="softmax"))
    
        model.compile(optimizer = RMSprop(lr = learning_rate),
                  loss = "categorical_crossentropy",
                  metrics = ['accuracy'])
    
        return model

In [19]:
model = model(0.0001)
print(model.summary())

ResourceExhaustedError: ignored

In [15]:
total_epochs = 10

history_fine = model.fit(train_batches,
                         epochs = total_epochs,
                         shuffle=True, 
                         initial_epoch=0,
                         validation_data = val_batches)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


35/35 [==============================] - 24s 675ms/step - loss: 1.4424 - accuracy: 0.5188 - val_loss: 5.2071 - val_accuracy: 0.4643
Epoch 2/10
35/35 [==============================] - 18s 516ms/step - loss: 1.0835 - accuracy: 0.5837 - val_loss: 1.1371 - val_accuracy: 0.5179
Epoch 3/10
35/35 [==============================] - 18s 516ms/step - loss: 1.2083 - accuracy: 0.6192 - val_loss: 1.5258 - val_accuracy: 0.5000
Epoch 4/10
35/35 [==============================] - 18s 519ms/step - loss: 0.9059 - accuracy: 0.6736 - val_loss: 1.4211 - val_accuracy: 0.5357
Epoch 5/10
35/35 [==============================] - 18s 519ms/step - loss: 0.8449 - accuracy: 0.6925 - val_loss: 1.3823 - val_accuracy: 0.5179
Epoch 6/10
35/35 [==============================] - 18s 522ms/step - loss: 0.9498 - accuracy: 0.6778 - val_loss: 0.6836 - val_accuracy: 0.5893
Epoch 7/10
35/35 [==============================] - 18s 517ms/step - loss: 0.9157 - accuracy: 0.6736 - val_loss: 1.1827 - val_accuracy: 0.4286
Epoch 8/10

In [ ]:
model.evaluate(test_batches)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2/2 [==============================] - 1s 311ms/step


[1.8604623079299927, 0.3928571343421936]

In [ ]:
print(model.metrics_names)

['loss', 'accuracy']


In [ ]:
!pip install -U ipython
!pip install tensorflowjs

     |████████████████████████████████| 788kB 3.5MB/s 
     |████████████████████████████████| 358kB 15.4MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.14.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


     |████████████████████████████████| 61kB 2.2MB/s 
     |████████████████████████████████| 92kB 5.3MB/s 
     |████████████████████████████████| 144.4MB 107kB/s 
     |████████████████████████████████| 256kB 38.6MB/s 
     |████████████████████████████████| 921kB 22.4MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=18e5d53ad36966105f0e39ec1b41bbca5b50599bae3ca424394e5fe0af0d7283
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: ipython 7.14.0 has requirement prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.14.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: prompt-toolkit 3.0.5
    Unins

In [ ]:
!mkdir model_85

In [ ]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "./model_85")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
!zip -r ./model_85.zip ./model_85

  adding: model_85/ (stored 0%)
  adding: model_85/group1-shard7of50.bin (deflated 7%)
  adding: model_85/group1-shard26of50.bin (deflated 7%)
  adding: model_85/group1-shard5of50.bin (deflated 7%)
  adding: model_85/group1-shard12of50.bin (deflated 7%)
  adding: model_85/group1-shard19of50.bin (deflated 7%)
  adding: model_85/group1-shard49of50.bin (deflated 7%)
  adding: model_85/group1-shard17of50.bin (deflated 7%)
  adding: model_85/group1-shard13of50.bin (deflated 7%)
  adding: model_85/group1-shard33of50.bin (deflated 7%)
  adding: model_85/group1-shard45of50.bin (deflated 7%)
  adding: model_85/group1-shard34of50.bin (deflated 7%)
  adding: model_85/group1-shard35of50.bin (deflated 7%)
  adding: model_85/group1-shard6of50.bin (deflated 7%)
  adding: model_85/group1-shard50of50.bin (deflated 7%)
  adding: model_85/group1-shard24of50.bin (deflated 7%)
  adding: model_85/group1-shard2of50.bin (deflated 7%)
  adding: model_85/group1-shard43of50.bin (deflated 7%)
  adding: model_85/g